In [ ]:
from google.colab import drive
import os
import pandas as pd
import torch

drive.mount('/content/gdrive/')
path = '/content/gdrive/Shared drives/16통계/data/'

movies = pd.read_csv(path + "navercombine.csv")
body = movies.copy()
# body = movies.loc['review']

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
movies.head()
#print(len(movies))

,Unnamed: 0,title,director,country,ganre,info,number,review,summary
0,0,남매의 여름밤,윤단비,한국,드라마,"방학 동안, 아빠와 함께 할아버지 집에서 지내게 된 남매 옥주와 동주, 그렇게 오...",1,반지하라도 ‘우리’집이 있었다. 이제는 ‘우리’집이 아닌 할아버지 댁으로 들어가...,ayekpaywutuluy eselphun tus kasumul wullinun k...
1,1,너의 이름은.,신카이 마코토,일본,"애니메이션, 드라마","아직 만난 적 없는 너를, 찾고 있어 천년 만에 다가오는 혜성 기적이 시작된...",2,부산국제영화제에서 처음 봤던 영화는 '너의 이름은' 이라는 애니였다. 친구가 표를 ...,inun kwake cipuli aynieyse sahoycek mwunceyey ...
2,2,기기괴괴 성형수,조경훈,한국,"공포, 스릴러",지금부터 기적의 성형수 사용 방법을 알려드리겠습니다. 본인의 얼굴을 거울로 확인해...,3,영화 기기괴괴 성형수를\r\n시사회로 보고왔어요\r\n\r\n 영화 기기괴괴 성형수...,
3,3,나이브스 아웃,라이언 존슨,미국,"미스터리, 스릴러",베스트셀러 미스터리 작가가 85세 생일에 숨진 채 발견된다. 그의 죽음의 원인을 ...,4,"나이브스 아웃 범인은 우리안에 있다 추리 스릴러\r\n\r\n나이브스 아웃, 줄거리...",haciman iyenghwauy cincca sicakun yenghwa hwup...
4,5,먼 훗날 우리,유약영,중국,"드라마, 멜로/로맨스","2007년 춘절, 귀향하는 기차에서 처음 만나 친구가 된 ‘린젠칭’(정백연)과 ‘팡...",6,"먼 훗날 우리\r\n감독\r\n유약영\r\n출연\r\n정백연, 주동우, 톈좡좡\r\...",-kuttay neyka yongki nayse cihacheley ollathas...


In [ ]:
from summarizer import Summarizer
model = Summarizer()

In [ ]:
if torch.cuda.is_available():
  print('o')
else:
  print('x')

x


### 이 부분


*유사도 코드*


In [ ]:
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/gdrive/')
path = '/content/gdrive/Shared drives/16통계/data/'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

def get_cosinesim(movie_number):
  bookdata = pd.read_csv(path + "bookSumm.csv")
  moviedata = pd.read_csv(path + "mSumm.csv")
  # bookdata, moviedata 합치기 위해 세 변수만 가져오기
  book_data = bookdata[['title', 'summary', 'isbn']]
  movie_data = moviedata[['title', 'summary', 'number']]
  tfidf = TfidfVectorizer(stop_words='english')
  recommend =  movie_data.loc[movie_data['number'] == movie_number]
  recommend.columns = ["title", "summary", 'isbn']
  #print(recommend)
  book_data = book_data.append(recommend)
  #print(book_data)
  book_data = book_data.reset_index()
  tfidf_matrix = tfidf.fit_transform(book_data['summary'])
  cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
  indices = pd.Series(book_data.index, index=book_data['summary']).drop_duplicates()
  return cosine_sim, indices

In [ ]:
def get_recommendations(movie_number):
    cosine_sim, indices = get_cosinesim(movie_number)
    # 선택한 영화의 number로부터 해당되는 인덱스 받아오기
    # book의 인덱스가 398까지 있음 맨 마지막에 영화 정보 추가됨
    idx = indices[399]
    # 모든 독서에 대해서 해당 영화와의 코사인 유사도 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))
    # 유사도에 따라 도서들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # 가장 유사한 5개의 도서 받아오기
    sim_scores = sim_scores[1:6]
    # 가장 유사한 5개의 도서의 인덱스 받아오기
    book_indices = [i[0] for i in sim_scores]
    # 가장 유사한 5개의 도서의 index를 리턴
    # isbn은 숫자가 꺠져서 출력돼서 index로 가져옴
    return book_data['index'].iloc[book_indices]

In [ ]:
idx = get_recommendations(1) # 영화데이터에서 number==1인 경우 유사한 도서의 인덱스 출력
isbn = []
for i in idx.index:
  isb = bookdata.loc[i, "isbn"]
  isbn.append(isb)
isbn

     title                                            summary  isbn
0  남매의 여름밤  ayekpaywutuluy eselphun tus kasumul wullinun k...     1
              title  ...           isbn
0         책 좀 빌려줄래?  ...  9791155812846
1          오늘부터의 세계  ...  9791157062010
2          김미경의 리부트  ...  9788901243665
3             더 플러스  ...  9791130630557
4    다시 춤추기 시작할 때까지  ...  9788932817651
..              ...  ...            ...
395    먹어서 병을 이기는 법  ...  9788965963868
396           백년 허리  ...  9788983717573
397    체리새우: 비밀글입니다  ...  9788954654753
398             페인트  ...  9788936456894
0           남매의 여름밤  ...              1

[400 rows x 3 columns]


[9791189604110, 9791170431015, 9791159036514, 9788936811532, 9791195522125]